In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [3]:
df.dropna(inplace=True)

In [4]:
df.shape

(714, 3)

In [5]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [6]:
X = df.iloc[:,1:3]
y = df.iloc[:,0]

In [7]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
x_train.head(2)

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542


In [9]:
clf = DecisionTreeClassifier()

In [10]:
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [11]:
pred = clf.predict(x_test)

In [12]:
print(accuracy_score(y_test,pred))

0.6223776223776224


In [13]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.6330985915492957

In [33]:
kbin_age = KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='kmeans')
kbin_fare = KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='kmeans')

# Create a ColumnTransformer
trf = ColumnTransformer(
    transformers=[
        ('first', kbin_age, [0]),
        ('second', kbin_fare, [1])
    ],
    remainder='passthrough'
)

# Fit and transform your data (X_train and X_test are your feature matrices)
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_discretization.py:239: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_discretization.py:239: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


In [34]:
print(trf.named_transformers_['first'].bin_edges_)

[array([ 0.42      ,  5.27754049, 11.74158654, 17.35243056, 21.55720029,
        26.64837093, 32.14464286, 37.41916667, 42.9025    , 48.1075    ,
        53.110625  , 58.090625  , 62.3875    , 67.72916667, 75.54166667,
        80.        ])                                                   ]


In [35]:
print(trf.named_transformers_['first'].bin_edges_)

[array([ 0.42      ,  5.27754049, 11.74158654, 17.35243056, 21.55720029,
        26.64837093, 32.14464286, 37.41916667, 42.9025    , 48.1075    ,
        53.110625  , 58.090625  , 62.3875    , 67.72916667, 75.54166667,
        80.        ])                                                   ]


In [37]:
output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf':x_train_trf[:,0],
    'fare':x_train['Fare'],
    'fare_trf':x_train_trf[:,1]
})

In [39]:
output['age_labels'] = pd.cut(x=x_train['Age'],
                             bins = trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=x_train['Fare'],
                             bins = trf.named_transformers_['second'].bin_edges_[0].tolist())

In [40]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
599,49.0,9.0,56.9292,4.0,"(48.108, 53.111]","(45.108, 60.257]"
43,3.0,0.0,41.5792,3.0,"(0.42, 5.278]","(32.498, 45.108]"
383,35.0,6.0,52.0000,4.0,"(32.145, 37.419]","(45.108, 60.257]"
269,35.0,6.0,135.6333,9.0,"(32.145, 37.419]","(123.564, 144.756]"
855,18.0,3.0,9.3500,0.0,"(17.352, 21.557]","(0.0, 11.665]"


In [41]:
clf = DecisionTreeClassifier()

In [42]:
clf.fit(x_train_trf,y_train)

DecisionTreeClassifier()

In [43]:
pred_trf = clf.predict(x_test)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [44]:
accuracy_score(y_test,pred_trf)

0.5104895104895105